In [1]:
# Step 1.1
# Import Libraries
import pandas as pd
import numpy as np

# To handle dates appropriately
from datetime import datetime

# Display options
pd.set_option('display.max_columns', None)


In [2]:
# Step 1.2
# Load data from your CSV file
data = pd.read_csv(r'V:\LIT\Can Kuvelet\DATA\SO_ISC_FORECAST.CSV', encoding='ISO-8859-1')

# Preview first 5 rows
data.head()


C:\Users\ckuvelet\AppData\Local\Temp\ipykernel_5972\1277522745.py:3: DtypeWarning: Columns (5,6,7,9,25,26,28,44) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(r'V:\LIT\Can Kuvelet\DATA\SO_ISC_FORECAST.CSV', encoding='ISO-8859-1')


,Customer ID,Customer Name,Sales Order/Proposal #,Date,Ship By,Proposal,Proposal Accepted,Closed,Quote #,Drop Ship,Ship to Name,Ship to Address-Line One,Ship to Address-Line Two,Ship to City,Ship to State,Ship to Zipcode,Ship to Country,Customer PO,Ship Via,Discount Amount,Displayed Terms,Sales Representative ID,Accounts Receivable Account,Accounts Receivable Amount,Sales Tax ID,Invoice Note,Note Prints After Line Items,Statement Note,Stmt Note Prints Before Ref,Internal Note,Number of Distributions,SO/Proposal Distribution,Quantity,Item ID,G/L Account,Unit Price,Tax Type,UPC / SKU,Weight,U/M ID,U/M No. of Stocking Units,Stocking Quantity,Stocking Unit Price,Amount,Job ID,Sales Tax Agency ID,Transaction Period,Transaction Number
0,ITLEON,Leoni Italy srl,150170,2/12/2015,4/17/2015,False,False,False,NaN,False,LEONI Italy srl,S.S. 10 Padana Ovest,17/19,Felizzano,AL,15023,ITALY,41025489,Ocean Freight,0.0,Prepaid,NaN,11101.0,7200.0,NaN,NaN,False,NaN,False,NaN,4.0,1.0,0.0,NaN,40101.0,0.00,1.0,NaN,0.0,NaN,1.0,0.0,0.00,0.0,NaN,NaN,0.0,13395.0
1,ITLEON,Leoni Italy srl,150170,2/12/2015,4/17/2015,False,False,False,NaN,False,LEONI Italy srl,S.S. 10 Padana Ovest,17/19,Felizzano,AL,15023,ITALY,41025489,Ocean Freight,0.0,Prepaid,NaN,11101.0,7200.0,NaN,NaN,False,NaN,False,NaN,4.0,5.0,0.0,NaN,40101.0,0.00,1.0,NaN,0.0,NaN,1.0,0.0,0.00,0.0,NaN,NaN,0.0,13395.0
2,ITLEON,Leoni Italy srl,150170,2/12/2015,4/17/2015,False,False,False,NaN,False,LEONI Italy srl,S.S. 10 Padana Ovest,17/19,Felizzano,AL,15023,ITALY,41025489,Ocean Freight,0.0,Prepaid,NaN,11101.0,7200.0,NaN,NaN,False,NaN,False,NaN,4.0,6.0,0.0,NaN,40101.0,0.00,1.0,NaN,0.0,NaN,1.0,0.0,0.00,0.0,NaN,NaN,0.0,13395.0
3,ITLEON,Leoni Italy srl,150170,2/12/2015,4/17/2015,False,False,False,NaN,False,LEONI Italy srl,S.S. 10 Padana Ovest,17/19,Felizzano,AL,15023,ITALY,41025489,Ocean Freight,0.0,Prepaid,NaN,11101.0,7200.0,NaN,NaN,False,NaN,False,NaN,4.0,2.0,40000.0,80.00JO.01238,40101.0,0.18,1.0,NaN,0.0,FT,1.0,40000.0,0.18,-7200.0,150170,NaN,0.0,13395.0
4,USRANI,RANIA,155707,10/16/2015,10/16/2015,False,False,False,NaN,False,RANIA,LA RAMBLA C/ NAVARRA,NaN,PONCE,PR,730,ISA,SOUTH BEACH3663-4146,Cust. Pickup,0.0,CHARGE CC EVERY MONTH,NaN,11201.0,5550.0,NaN,NaN,False,NaN,False,NaN,1.0,1.0,100.0,NaN,40180.0,55.50,1.0,NaN,0.0,NaN,1.0,100.0,55.50,-5550.0,155707,NaN,0.0,15116.0


In [3]:
# Step 1.3
# Check initial row count
print(f"Initial Rows: {data.shape[0]}")

# Remove rows with Quantity = 0 or Quantity is null
data_clean = data[data['Quantity'].notnull() & (data['Quantity'] != 0)]

# Remove rows with null Item ID
data_clean = data_clean[data_clean['Item ID'].notnull()]

# Keep only rows where Item ID starts with '61' and has length of 6
data_clean = data_clean[
    data_clean['Item ID'].astype(str).str.startswith('61') &
    (data_clean['Item ID'].astype(str).str.len() == 6)
]

# Check rows after cleaning
print(f"Rows after cleaning: {data_clean.shape[0]}")


Initial Rows: 486247
Rows after cleaning: 322745


In [4]:
# Step 1.4
# Convert 'Date' to datetime
data_clean['Date'] = pd.to_datetime(data_clean['Date'], errors='coerce')

# Filter data from 2023 onwards
data_2023 = data_clean[data_clean['Date'] >= '2023-01-01'].copy()

# Check rows after filtering
print(f"Rows from 2023 onwards: {data_2023.shape[0]}")


Rows from 2023 onwards: 134174


In [5]:
#Step 1.5
# Create Year-Month column
data_2023['YearMonth'] = data_2023['Date'].dt.to_period('M')

# Aggregate monthly quantities per SKU
monthly_sku_data = data_2023.groupby(['YearMonth', 'Item ID']).agg({
    'Quantity': 'sum'
}).reset_index()

# Rename columns clearly
monthly_sku_data.columns = ['YearMonth', 'Item_ID', 'Monthly_Quantity']

# Check aggregated data
monthly_sku_data.head()


,YearMonth,Item_ID,Monthly_Quantity
0,2023-01,610009,6.0
1,2023-01,610017,36.0
2,2023-01,610021,4.0
3,2023-01,610029,25.0
4,2023-01,610033,150.0


In [6]:
# Step 1.6
# Get total quantities per SKU to identify top SKUs
total_qty_per_sku = monthly_sku_data.groupby('Item_ID')['Monthly_Quantity'].sum().reset_index()

# Sort SKUs by descending quantity
top_skus = total_qty_per_sku.sort_values(by='Monthly_Quantity', ascending=False).head(500)
top_skus = top_skus.rename(columns={'Monthly_Quantity': 'Total Sales Quantity'})

# Check top SKUs
top_skus.head(10)


,Item_ID,Total Sales Quantity
1573,612820,5746.0
2189,613700,4698.0
1209,612244,3969.0
1747,613070,3769.0
478,610830,3527.0
2145,613630,3403.0
310,610562,3401.0
1924,613315,3098.0
777,611454,3077.0
1447,612620,2892.0


In [7]:
# Step 1.7
# Top 500 SKUs data
top_sku_data = monthly_sku_data[monthly_sku_data['Item_ID'].isin(top_skus['Item_ID'])]

# Remaining SKUs data
remaining_sku_data = monthly_sku_data[~monthly_sku_data['Item_ID'].isin(top_skus['Item_ID'])]

# Preview data
print(f"Top SKUs data rows: {top_sku_data.shape[0]}")
print(f"Remaining SKUs data rows: {remaining_sku_data.shape[0]}")
print(f"All SKUs data rows: {monthly_sku_data.shape[0]}")

Top SKUs data rows: 12821
Remaining SKUs data rows: 27311
All SKUs data rows: 40132


In [8]:
# Step 1.8
# Export datasets
top_sku_data.to_csv('top_sku_monthly.csv', index=False)
remaining_sku_data.to_csv('remaining_sku_monthly.csv', index=False)
monthly_sku_data.to_csv('all_sku_monthly.csv', index=False)